In [16]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle


In [17]:
page = requests.get('https://maldita.es/malditobulo/')

In [18]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [19]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [20]:
'''
tags = soup.find_all('article', class_='card card-item half')

todo = list(tags)
#print(todo)

#Comienzo a procesar el texto html y me quedo solo con los títulos

elementos = []
for articulo in todo:
    #print(articulo)

    if (articulo.get_text() != '\n'):
        #print(articulo.get_text().split('\n'))
        elementos.append(articulo.get_text().strip().split('\n'))
    
#print(elementos)

lista_titulos = []
for e in elementos:

    lista_titulos.append(e[-1].strip())
    

print(len(elementos))
print(len(lista_titulos))
print(lista_titulos)

lista_titulos
'''

"\ntags = soup.find_all('article', class_='card card-item half')\n\ntodo = list(tags)\n#print(todo)\n\n#Comienzo a procesar el texto html y me quedo solo con los títulos\n\nelementos = []\nfor articulo in todo:\n    #print(articulo)\n\n    if (articulo.get_text() != '\n'):\n        #print(articulo.get_text().split('\n'))\n        elementos.append(articulo.get_text().strip().split('\n'))\n    \n#print(elementos)\n\nlista_titulos = []\nfor e in elementos:\n\n    lista_titulos.append(e[-1].strip())\n    \n\nprint(len(elementos))\nprint(len(lista_titulos))\nprint(lista_titulos)\n\nlista_titulos\n"

In [21]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [22]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url}
    
    return article

In [23]:
lista_enlaces = []
suficiente = True
n_pag = 1
n_noticias = 1#Las noticias van en bloques de 24. La carga computacional aumentaria mucho si quisiera que fuera mas preciso
                #Pero para este caso de uso y estudio no es necesario.

#Cargo las URLS.
print('Cargando URLs...')

while (len(lista_enlaces) <= n_noticias-1 and suficiente):
    link =  "https://maldita.es/malditobulo/page/" + str(n_pag) + "/"
    arrayurls= get_urls(link, lista_enlaces)
    n_pag = n_pag + 1

    if (n_pag >=10):
        suficiente=False
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/05/25/poema-epidemia-peste-1800/', 'https://maldita.es/malditobulo/2020/05/25/no-telefonica-no-esta-sorteando-productos-apple-y-samsung-por-su-aniversario-es-phishing/', 'https://maldita.es/malditobulo/2020/05/25/correo-seguridad-social-reembolso-phishing/', 'https://maldita.es/malditobulo/2020/05/25/explosion-bote-gel-hidoralcoholico-coche-malaga-sevilla/', 'https://maldita.es/malditobulo/2020/05/25/ambulancia-manifestacion-vox-santander/', 'https://maldita.es/malditobulo/2020/05/25/foto-traje-antirradiacion-tecnologia-5g-riesgo-salud/', 'https://maldita.es/malditobulo/2020/05/25/bonos-gratis-combustible-timo-phishing-repsol-shell-pemex/', 'https://maldita.es/malditaciencia/2020/05/25/cadena-italia-cura-coronavirus-autopsias-prohibidas/', 'https://maldita.es/malditobulo/2020/05/25/no-hay-componente-marca-cov-19-circuitos-electronicos-torres-5g-video/', 'https://maldita.es/malditobulo/2020/05/24/coronavirus-bacteria-5g-autopsias

In [24]:
lista_articulos

[{'Title': 'No, este poema no fue escrito durante la epidemia de peste de 1800: es actual y está inspirado en la pandemia de coronavirus',
  'Publication_date': '25/05/2020',
  'Content': 'Está circulando un poema firmado por K. O\'Meara que comienza con el verso "y la gente se quedó en casa", supuestamente escrito "hace 220 años aproximadamente", durante la epidemia de peste que hubo en 1.800. Es un bulo. El título del poema es "In the Time of Pandemic", está inspirado en la actual pandemia de coronavirus y la autora es Kitty O\'Meara, quien lo publicó en en su blog el 16 de marzo.K. O\'Meara es Kitty O\'Meara, escritora que ha trabajado como maestra y asistente espiritual en hospitales y hospicios, tal como se define a sí misma en su blog The Daily Round. El pasado 16 de marzo publicó en inglés el poema llamado "In the Time of Pandemic" que más tarde se viralizó como si hubiese sido escrito en el siglo XIX.O\'Meara habló sobre el poema en una entrevista en The Oprah Magazine en marzo

In [25]:
from datetime import date


date = date.today().strftime("%d-%m-%Y")


filename = 'Maldita_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

Maldita_FN_25-05-2020


In [26]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'No, este poema no fue escrito durante la epidemia de peste de 1800: es actual y está inspirado en la pandemia de coronavirus',
  'Publication_date': '25/05/2020',
  'Content': 'Está circulando un poema firmado por K. O\'Meara que comienza con el verso "y la gente se quedó en casa", supuestamente escrito "hace 220 años aproximadamente", durante la epidemia de peste que hubo en 1.800. Es un bulo. El título del poema es "In the Time of Pandemic", está inspirado en la actual pandemia de coronavirus y la autora es Kitty O\'Meara, quien lo publicó en en su blog el 16 de marzo.K. O\'Meara es Kitty O\'Meara, escritora que ha trabajado como maestra y asistente espiritual en hospitales y hospicios, tal como se define a sí misma en su blog The Daily Round. El pasado 16 de marzo publicó en inglés el poema llamado "In the Time of Pandemic" que más tarde se viralizó como si hubiese sido escrito en el siglo XIX.O\'Meara habló sobre el poema en una entrevista en The Oprah Magazine en marzo

In [27]:
'''
page = requests.get('https://maldita.es/malditaciencia/2020/05/21/cadena-italia-cura-coronavirus-autopsias-prohibidas/')

soup3 = BeautifulSoup(page.content, 'html.parser')

title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
print(title)

date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
print(date)

content_list = []
content = soup3.find("div", class_="article-body")
content2 = content.find_all("p", class_="")

for e in content2:
    content_list.append(e.get_text().replace("\xa0", " "))

content = '\n'.join(content_list)
print(content)
'''

'\npage = requests.get(\'https://maldita.es/malditaciencia/2020/05/21/cadena-italia-cura-coronavirus-autopsias-prohibidas/\')\n\nsoup3 = BeautifulSoup(page.content, \'html.parser\')\n\ntitle = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()\nprint(title)\n\ndate = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip(\'\n\')\nprint(date)\n\ncontent_list = []\ncontent = soup3.find("div", class_="article-body")\ncontent2 = content.find_all("p", class_="")\n\nfor e in content2:\n    content_list.append(e.get_text().replace("\xa0", " "))\n\ncontent = \'\n\'.join(content_list)\nprint(content)\n'

In [31]:
#article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}
url = 'https://maldita.es/malditobulo/2020/05/25/explosion-bote-gel-hidoralcoholico-coche-malaga-sevilla/'
page = requests.get(url)

soup3 = BeautifulSoup(page.content, 'html.parser')

title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
#print(title)

date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
#print(date)

content_list = []
body = soup3.find("div", class_="article-body")
body2 = body.find_all("p", class_="")

for e in body2:
    content_list.append(e.get_text().replace("\xa0", " "))

content = ''.join(content_list)

article = {"Title": title, "Publication_date": date, "Content": content, "URL": url}

article

{'Title': 'No, esta foto no es de una explosión causada por dejar un bote de gel hidroalcohólico en un coche de la Policía Nacional de Málaga y es improbable que algo así pueda ocurrir',
 'Publication_date': '25/05/2020',
 'Content': 'Nos habéis preguntado a través de nuestro servicio de WhatsApp y por La Buloteca si esta imagen es de la "explosión de un bote de hidrogel" dejado en el interior de un coche expuesto al sol en Málaga o en Sevilla es real. El contenido que se mueve en Málaga afirma que se trata de un vehículo de la Policía Nacional, mientras que en Sevilla aseguran que es de la ONG S.O.S bomberos sin fronteras. Es un bulo. Al fondo de la imagen, que ha circulado también en otros países, se puede ver un letrero en portugués y la Policía Nacional de Málaga dice que "en la provincia de Málaga" no se ha registrado "ningún suceso con vehículos del CNP". El mensaje habla de la ONG S.O.S bomberos sin fronteras pero no existe ninguna ONG con ese nombre, sino la ONG S.O.S. Ayuda Si